In [ ]:
#Wenn man den typen bestimmte hat, sollte man zunächst alle Pflicht-Felder des Bibtex-Eintrages raussuchen und bestimmen.
#Dazu die Pflichtattribute in einer Schleife durchgehen.

In [1]:
import torch
import pandas as pd
torch.cuda.is_available()

True

In [45]:
from transformers import pipeline
ner_tagger = pipeline("ner", aggregation_strategy="simple")
text="Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948"
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)

df_PER = df_outputs[df_outputs["entity_group"] == "PER"]

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

In [46]:
df_PER

,entity_group,score,word,start,end
0,PER,0.996079,Hinton,0,6
1,PER,0.998951,G,8,9
2,PER,0.994906,Bengio,12,18
3,PER,0.998544,Y,20,21
4,PER,0.963430,LeCun,27,32
5,PER,0.998110,Y,34,35
8,PER,0.949492,J. Smith,91,99
9,PER,0.992646,J. Doe,102,108


In [24]:
startIndexPER = df_PER["start"].iloc[0]
endIndexPER = df_PER["end"].iloc[-1]

In [17]:
import pandas as pd

# Beispiel DataFrame erstellen
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Zugriff auf die erste Zeile
erste_zeile = df.iloc[0]
print(erste_zeile)

A    1
B    4
C    7
Name: 0, dtype: int64


In [42]:
#Test, um Datum Zuverlässig erkannt wird

import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("Hinton, G., Bengio, Y. and LeCun, Y. (2021). Deep Learning for Artificial Intelligence. In J. Smith & J. Doe (Eds.), Proceedings of the IEEE International Conference on Neural Networks (Vol. 1, Issue 5, pp. 100–120). IEEE Press. https://doi.org/10.1109/ICNN.2021.9483948")
for ent in doc.ents:
  print(ent, ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_)

Hinton 0 6 GPE
G. 8 10 GPE
Bengio 12 18 PERSON
Y. 20 22 PERSON
Y. 34 36 PERSON
2021 38 42 DATE
Deep Learning for Artificial Intelligence 0 41 ORG
J. Smith & J. Doe (Eds 3 25 ORG
Proceedings of the IEEE International Conference on Neural Networks 29 96 ORG
1 103 104 CARDINAL
5 112 113 DATE
100–120 119 126 CARDINAL
IEEE Press 0 10 ORG
https://doi.org/10.1109/ICNN.2021.9483948 0 41 GPE


@book{advanced_physics, author = {Stephen Hawking and Brian Greene and Maria Trump and Susan Soy}, title = {Advanced Concepts in Theoretical Physics}, publisher = {Cambridge University Press}, year = {2025}, edition = {3}, volume = {1}, series = {Advanced Studies in Physics}, address = {Cambridge, UK}, month = {May}, note = {A seminal work in the field of theoretical physics}, isbn = {978-0-521-76948-0}, doi = {10.1017/example.book.2025}, url = {https://www.cambridge.org/advanced_physics}, annote = {Widely cited in the physics community}, abstract = {This book explores cutting-edge theories and concepts in theoretical physics...}, keywords = {Theoretical Physics, Quantum Mechanics, String Theory}, language = {English}, price = {75.00}, size = {600 pages}, lccn = {2020934576}, mrnumber = {MR3070071} }

@article{quantum_entanglement,
  author        = {Albert Einstein and Boris Podolsky and Nathan Rosen},
  title         = {Can Quantum-Mechanical Description of Physical Reality Be Considered Complete?},
  journal       = {Physical Review},
  year          = {1935},
  volume        = {47},
  number        = {10},
  pages         = {777-780},
  month         = {May},
  note          = {EPR Paradox paper, fundamental for quantum mechanics},
  doi           = {10.1103/PhysRev.47.777},
  url           = {https://journals.aps.org/pr/abstract/10.1103/PhysRev.47.777},
  abstract      = {In this paper, the authors discuss the EPR paradox and challenge the completeness of quantum mechanics...},
  keywords      = {Quantum Mechanics, EPR Paradox, Physical Reality},
  language      = {English},
  publisher     = {American Physical Society}
}

@inproceedings{deep_learning,
  author        = {Geoffrey Hinton and Yoshua Bengio and Yann LeCun},
  title         = {Deep Learning for Artificial Intelligence},
  booktitle     = {Proceedings of the IEEE International Conference on Neural Networks},
  year          = {2021},
  editor        = {Jane Smith and John Doe},
  volume        = {1},
  number = {5},
  series        = {Advances in Neural Information Processing},
  pages         = {100-120},
  address       = {Montreal, Canada},
  month         = {December},
  organization  = {IEEE},
  publisher     = {IEEE Press},
  note          = {Keynote paper on recent advancements in deep learning},
  isbn          = {978-1-5386-4637-1},
  doi           = {10.1109/ICNN.2021.9483948},
  url           = {https://ieeexplore.ieee.org/document/9483948},
  annote        = {A seminal work on how deep learning transforms AI},
  abstract      = {This paper explores cutting-edge deep learning techniques and their impact on the development of artificial intelligence...},
  keywords      = {Deep Learning, Artificial Intelligence, Neural Networks},
  language      = {English}
}


@incollection{quantum_computation,
  author        = {Michael A. Nielsen and Isaac L. Chuang},
  title         = {Quantum Computation and Quantum Information},
  booktitle     = {Handbook of Quantum Information Science},
  publisher     = {Springer},
  year          = {2026},
  editor        = {Charles H. Bennett and David P. DiVincenzo},
  volume        = {4},
  series        = {Quantum Science and Technology},
  chapter       = {10},
  pages         = {250-300},
  address       = {Berlin, Germany},
  month         = {October},
  note          = {A comprehensive overview of the fundamentals of quantum computation},
  isbn          = {978-3-540-88702-7},
  doi           = {10.1007/springerreference_303198},
  url           = {https://www.springer.com/gp/book/9783540887027},
  annote        = {Essential reading for researchers entering the field of quantum information},
  abstract      = {This chapter delves into the principles of quantum computing, offering an accessible yet thorough introduction...},
  keywords      = {Quantum Computing, Quantum Information, Computational Models},
  language      = {English},
  price         = {45.00},
  size          = {50 pages}
}


In [11]:
#RegEx zum FInden der Seiten
pageFinder = ", (?:pp\.? )?\d+-\d+,"
doi1 = "https:\/\/doi\.org"
doi2 = "DOI:"

#if havard und book:
volume = ", \d+,"
number = ", \d+,"
edition1 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) ed\."
edition2 = "(?:[1-9]\d*th|11th|12th|13th|[1-9]\d*(?:st|nd|rd)) edn\."

#if APA or Havard then
volume = "\(Vol\. \d+" #für Volume und number noch als Bedingung, dass es nach Titel stehen muss, also Index > Titel-Index
number = "Issue \d+"
yearApaHavard = "\(\d{4}\)"
editorApaHavard = "\((Eds\.|Eds|Ed|ed|eds\.)\)"

#editor bei anderen
editorIEEEMLAACM = "(Eds\.|Eds|Ed|ed|eds\.)"

#Beachte: Editor und Edition werden oft mit gleichen Abkürzungen versehen, auch im gleichen Stil! Nur Edition wird oft
#klein geschrieben. Daher zustzlich prüfen, ob NER das als Person erkennt.

#Beachte: ieee und mla trennen mit , und nicht mit .
#if ieee and mla
volumeIeeeMla = ", vol\. \d+" 
noIeeeMla = ", no\. \d+"

etAl = "et al."

yearACM = "\. \d{4}\."

#Man sollte zunächst die Felder extrahieren, die zu 100% sicher erkennen kann wie Volume oder Number, doi, Autoren...-> 
#Titel, booktitle, series und journal danach

#man sollte eine Überdeckungsprüfung machen: Also Bspw. sagt Space im Bereich 25-50 ist eine Orga, Regex sagt, 
#dort ist ein (Eds.)
#zu finden und Huggingface sagt, von 25-45 ist was. Dann sollte der minimalteste Wert und maximalste Wert 
#genommen werden und so lange nach links und analog nach rechts gehen, bis wieder ein Punkt kommt.
# Da erkannte Felder aus dem String entfernt werden, muss der String am Ende leer sein. 
#Daher sollten zunächste die Dinge ausgeschnitten werden wo der Algo sich am sichersten ist
#TODO: Grundsätzliche Reihenfolge der BibTex-Einträge ermitteln.
#TODO: Für jede Extrahierung eines Feldes eine eigene Unterroutine schreiben? -> In der Oberschleife sollte Style sein





In [12]:
import re

txt = "Tpp. 250–300). Springer. https://doi.org/10.1007/springerreference_303198"
x = re.search(doi1, txt)
startIndex = x.start()
endIndex = x.end()

print("The first white-space character is located in position:", x.start()) 


The first white-space character is located in position: 25


In [8]:
listRegEx = [pageFinder, doi]
for regex in listRegEx:
    x = re.search(regex, txt)
    if x:
        startIndex = x.start()
        endIndex = x.end()
        print(startIndex)

25


Test Parser APA + Book

In [48]:

ner_tagger = pipeline("ner", aggregation_strategy="simple")
text="Hawking, S., Greene, B., Trump, M., Soy, S. (2025). Advanced Concepts and John Miller in Theoretical Physics (3rd ed., Vol. 1). Cambridge University Press. https://doi.org/10.1017/example.book.2025"
outputs = ner_tagger(text)
df_outputs = pd.DataFrame(outputs)

df_PER = df_outputs[df_outputs["entity_group"] == "PER"]
df_PER_len = len(df_PER.index)
amountAuhtor = df_PER_len / 2
evenAuthorAmount = (amountAuhtor % 2) == 0
print("evenAuthorAmount: {0}".format(evenAuthorAmount))


onlyOneAuhtor = False   
if df_PER_RowCount == 2:
    onlyOneAuhtor = True
print("onlyOneAuhtor: {0}".format(onlyOneAuhtor))

print(df_PER)
startIndexPER = df_PER["start"].iloc[0]
endIndexPER = df_PER["end"].iloc[-1] + 1

#prüfe, wo der letzte Autor in der Autorenkette ist, um Namen auszuschließen, die im Titel sein können
#vorher prüfen, wo et al. steht, weil so erkennt man ende sicher
#zusäzich kann man noch prüfen, wo das erst "and" oder "&" kommt
authorWithAnd = False
indexStartLastAuthorInField = 0
indexEndPenultimateAuthorInField = 0
if not onlyOneAuhtor:
    for index in df_PER.index.values.tolist():
        if index < (df_PER_len - 1):
            if (df_PER["start"].iloc[index + 1] - df_PER["end"].iloc[index] > 2) \
            and (df_PER["start"].iloc[index + 1] - df_PER["end"].iloc[index] < 8):
                indexLastAuthorInField = df_PER["start"].iloc[index + 1]
                indexEndPenultimateAuthorInField = df_PER["end"].iloc[index] 
                break

    authors = text[startIndexPER:endIndexPER]
    indexAndInAuthors = authors.find(" and ")
    if indexAndInAuthors == -1:
        print("kein and")
        indexAndInAuthors = authors.find(" & ")
        if indexAndInAuthors == -1:
            print("kein and")
        
    if indexAndInAuthors > 0 and df_PER_len > 2 and df_PER["end"].iloc[-3] < indexAndInAuthors:
        print("es liegen wahrscheinlich Autoren mit und vor")
        if indexLastAuthorInField > 0 and \
        indexAndInAuthors > indexEndPenultimateAuthorInField and indexAndInAuthors < indexLastAuthorInField:
        print("es liegen höchstwahrscheinlich Autoren mit und vor")
        authorWithAnd = True
    
    #Nun Prüfen, ob der start des letzten Autors direkt hinter dem Index vom and kommt
    
if not authorWithAnd:
    print("Keine Autoren mit und")

authors = authors.replace(", ",",")
#wenn countKomma == df_PER_len - 1, dann liegt sowas vor: Hawking, S., Greene, B., Trump, M., & Soy, S.

authors = text[startIndexPER:endIndexPER]

countKomma = authors.count(",")
authors = authors.replace("&", "")
authors = authors.replace("and", "")
authors = authors.replace("et al.", "")
authorsList = authors.split(",")

surenameFirst = authorsList[0].strip().endswith(".") #Dann stehen Vornamen wohl zuerst.

maxRangeIndex = len(authorsList)-1
if not surenameFirst:
    for i in range(0, maxRangeIndex , 2):
        authorsList[i], authorsList[i+1] = authorsList[i+1].strip(), authorsList[i].strip()
    print(authorsList) 
authors = "author = {"
for i in range(0, maxRangeIndex , 2):
    authors = authors + " " + authorsList[i] + " " + authorsList[i+1]
    print(i)
    if i < (maxRangeIndex - 2):
        authors = authors + " and"
authors = authors + "}"
print(authors)
    

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\larsl\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

evenAuthorAmount: False
onlyOneAuhtor: False
  entity_group     score         word  start  end
0          PER  0.990826      Hawking      0    7
1          PER  0.998506            S      9   10
2          PER  0.999762       Greene     13   19
3          PER  0.998537            B     21   22
4          PER  0.999691        Trump     25   30
5          PER  0.997559            M     32   33
6          PER  0.985108          Soy     36   39
7          PER  0.998687            S     41   42
8          PER  0.993428  John Miller     74   85
es liegen wahrscheinlich Autoren mit und vor
['S.', 'Hawking', 'B.', 'Greene', 'M.', 'Trump', 'S. (2025). Advanced Concepts  John Miller', 'Soy']
0
2
4
6
author = { S. Hawking and B. Greene and M. Trump and S. (2025). Advanced Concepts  John Miller Soy}
